In [ ]:
#!pip install praw
#import praw
#from textblob import TextBlob
#import pandas as pd

# Credentials
#reddit = praw.Reddit(client_id='UEJ25LF2qfVqzplDerE3tw',
#                     client_secret='lS3wj-eQEAdXgJVGHhI3xzQjBxVq6Q',
#                     user_agent='SentimentAnalysis271197')

#subreddit_name = "nike"

# Number of posts to retrieve
#num_posts = 10000000

# Retrieve the top posts from the subreddit
#subreddit = reddit.subreddit(subreddit_name)

#recent_posts = subreddit.new(limit=num_posts)

#top_posts = subreddit.top(limit=num_posts)

# Initialize lists to store data
#titles = []
#scores = []
#urls = []
#sentiments = []

# Process and analyze sentiment for the posts
#for post in recent_posts:
#    title = post.title
#    score = post.score
#    url = post.url

    # Perform sentiment analysis using TextBlob
#    analysis = TextBlob(title)
#    sentiment = analysis.sentiment.polarity

    # Append data to lists
#    titles.append(title)
#   scores.append(score)
#    urls.append(url)
#    sentiments.append(sentiment)

    # Print sentiment analysis result
#    if sentiment > 0:
#        print("Sentiment: Positive")
#    elif sentiment == 0:
#        print("Sentiment: Neutral")
#    else:
#        print("Sentiment: Negative")

#    print("-" * 50)

# Create a pandas DataFrame
#data = {
#    'Title': titles,
#    'Score': scores,
#   'URL': urls,
#    'Sentiment': sentiments
#}

#df = pd.DataFrame(data)

# Print the DataFrame
#print(df)

#!pip install praw
#from textblob import TextBlob
#import pandas as pd
#import praw

#def get_posts_data(subreddit_name, num_posts):
    # Credentials
#    reddit = praw.Reddit(client_id='UEJ25LF2qfVqzplDerE3tw',
#                         client_secret='lS3wj-eQEAdXgJVGHhI3xzQjBxVq6Q',
#                         user_agent='SentimentAnalysis271197')

    # Retrieve the top posts from the subreddit
#    subreddit = reddit.subreddit(subreddit_name)

#    posts = subreddit.new(limit=num_posts)

    # Initialize lists to store data
#    titles = []
#    scores = []
#    urls = []
#    sentiments = []

    # Process and analyze sentiment for the posts
#    for post in posts:
#        title = post.title
#        score = post.score
#        url = post.url

        # Perform sentiment analysis using TextBlob
#        analysis = TextBlob(title)
#        sentiment = analysis.sentiment.polarity

        # Append data to lists
#        titles.append(title)
#        scores.append(score)
#        urls.append(url)
#        sentiments.append(sentiment)

        # Print sentiment analysis result
#        if sentiment > 0:
#            print("Sentiment: Positive")
#        elif sentiment == 0:
#            print("Sentiment: Neutral")
#        else:
#            print("Sentiment: Negative")

#        print("-" * 50)

    # Create a pandas DataFrame
#    data = {
#        'Title': titles,
#        'Score': scores,
#       'URL': urls,
#        'Sentiment': sentiments
#    }

#    df = pd.DataFrame(data)
#    return df

#subreddit_name = "nike"
#num_posts = 10000  # You can change this value to get more or fewer posts

#df = get_posts_data(subreddit_name, num_posts)

# Print the DataFrame
#print(df)

#Batchwise request to get more that 1000 posts
#!pip install pr
!pip install praw
!pip install mlflow==1.21.0
!pip install h2o
!pip uninstall pydantic -y
!pip install "pydantic==1.*"
import os
import mlflow
import joblib
import praw
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
import h2o
import nltk
from h2o.automl import H2OAutoML
from textblob import TextBlob
from sklearn.metrics import accuracy_score
import joblib

h2o.init()
nltk.download('stopwords')

def log_model_params(params):
    for key, value in params.items():
        mlflow.log_param(key, value)

def log_model_metrics(accuracy):
    mlflow.log_metric("accuracy", accuracy)

def train_lstm_model(X_train_lstm, y_train_lstm, X_val_lstm, y_val_lstm, vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(LSTM(128))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_lstm, y_train_lstm, validation_data=(X_val_lstm, y_val_lstm), epochs=5, batch_size=32)
    return model

# Define classifier_nb at a higher scope
classifier_nb = None
# Define classifier_svm at a higher scope
classifier_svm = None
classifier_lr = None

def get_posts_data(subreddit_name, num_posts_train, num_posts_predict):
    # Credentials
    reddit = praw.Reddit(client_id='UEJ25LF2qfVqzplDerE3tw',
                         client_secret='lS3wj-eQEAdXgJVGHhI3xzQjBxVq6Q',
                         user_agent='SentimentAnalysis271197')

    # Retrieve the posts from the subreddit
    subreddit = reddit.subreddit(subreddit_name)

    # Check the number of available posts in the subreddit
    posts = subreddit.new(limit=num_posts_predict)  # Retrieve most recent posts
    posts = list(posts)
    num_available_posts = len(posts)
    if num_posts_train > num_available_posts:
        num_posts_train = num_available_posts

    # Initialize lists to store data
    titles = []
    sentiments = []
    post_dates = []

    # Process and analyze sentiment for the training posts
    batch_data = []
    batch_size = 100
    num_batches = num_posts_train // batch_size

    for i in range(num_batches):
        batch_posts = posts[i * batch_size: (i + 1) * batch_size]
        for post in batch_posts:
            title = post.title.lower()
            try:
                sentiment = TextBlob(title).sentiment.polarity
            except Exception as e:
                print(f"Error analyzing sentiment for title: {title}. Setting neutral sentiment.")
                sentiment = 0.0  # Set neutral sentiment
            batch_data.append((title, sentiment, post.created_utc))

    if batch_data:
        titles, sentiments, post_dates = zip(*batch_data)

    if not titles:
        raise ValueError("No valid titles found. Check the input data.")

    # Process and analyze sentiment for the prediction posts
    titles_predict = [post.title.lower() for post in posts]
    post_dates_predict = [post.created_utc for post in posts]

    # Convert sentiment to categorical labels based on thresholds (e.g., -0.2, 0.2)
    sentiment_labels = ['Negative' if s < -0.2 else 'Positive' if s > 0.2 else 'Neutral' for s in sentiments]

    # Create a pandas DataFrame for training data
    train_data = {
        'Title': titles,
        'Sentiment': sentiment_labels,
        'Post Date': pd.to_datetime(post_dates, unit='s')
    }
    df_train = pd.DataFrame(train_data)

    # Preprocess the text data
    vectorizer = TfidfVectorizer(stop_words=None)  # Adjust the stop words
    X_train = vectorizer.fit_transform(df_train['Title'])
    y_train = df_train['Sentiment']

    # Split the data into training set and validation set for Naive Bayes
    X_train_nb, X_val_nb, y_train_nb, y_val_nb = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )

    global classifier_nb  # Add this line to indicate you're using the global variable

    # Train the Naive Bayes classifier
    classifier_nb = MultinomialNB()
    classifier_nb.fit(X_train_nb, y_train_nb)

    # Predict sentiment for the validation set using Naive Bayes
    y_val_pred_nb = classifier_nb.predict(X_val_nb)

    # Calculate accuracy for Naive Bayes
    accuracy_nb = accuracy_score(y_val_nb, y_val_pred_nb)

    # Preprocess the prediction data
    X_predict = vectorizer.transform(titles_predict)

    # Predict sentiment for the prediction data using Naive Bayes
    y_predict_nb = classifier_nb.predict(X_predict)

    # Create a pandas DataFrame for prediction data using Naive Bayes
    predict_data_nb = {
        'Title': titles_predict,
        'Sentiment': y_predict_nb,
        'Post Date': pd.to_datetime(post_dates_predict, unit='s')
    }
    df_predict_nb = pd.DataFrame(predict_data_nb)

    # Combine the training and prediction DataFrames using Naive Bayes
    df_combined_nb = pd.concat([df_train, df_predict_nb], ignore_index=True)

    # Split the data into training set and validation set for SVM
    X_train_svm, X_val_svm, y_train_svm, y_val_svm = train_test_split(
        X_train, y_train, test_size=0.2, random_state=0
    )

    # Train the SVM classifier
    global classifier_svm  # Add this line to indicate you're using the global variable
    classifier_svm = SVC()
    classifier_svm.fit(X_train_svm, y_train_svm)

    # Predict sentiment for the validation set using SVM
    y_val_pred_svm = classifier_svm.predict(X_val_svm)

    # Calculate accuracy for SVM
    accuracy_svm = accuracy_score(y_val_svm, y_val_pred_svm)

    # Predict sentiment for the prediction data using SVM
    y_predict_svm = classifier_svm.predict(X_predict)

    # Create a pandas DataFrame for prediction data using SVM
    predict_data_svm = {
        'Title': titles_predict,
        'Sentiment': y_predict_svm,
        'Post Date': pd.to_datetime(post_dates_predict, unit='s')
    }
    df_predict_svm = pd.DataFrame(predict_data_svm)

    # Combine the training and prediction DataFrames using SVM
    df_combined_svm = pd.concat([df_train, df_predict_svm], ignore_index=True)

    # Train the Logistic Regression classifier
    global classifier_lr
    classifier_lr = LogisticRegression()
    classifier_lr.fit(X_train, y_train)

    # Predict sentiment for the validation set using Logistic Regression
    y_val_pred_lr = classifier_lr.predict(X_val_nb)

    # Calculate accuracy for Logistic Regression
    accuracy_lr = accuracy_score(y_val_nb, y_val_pred_lr)

    # Predict sentiment for the prediction data using Logistic Regression
    y_predict_lr = classifier_lr.predict(X_predict)

    # Create a pandas DataFrame for prediction data using Logistic Regression
    predict_data_lr = {
        'Title': titles_predict,
        'Sentiment': y_predict_lr,
        'Post Date': pd.to_datetime(post_dates_predict, unit='s')
    }
    df_predict_lr = pd.DataFrame(predict_data_lr)

    # Combine the training and prediction DataFrames using Logistic Regression
    df_combined_lr = pd.concat([df_train, df_predict_lr], ignore_index=True)

    # Train the LSTM model
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(df_combined_nb['Title'])

    X_train_lstm = tokenizer.texts_to_sequences(df_combined_nb['Title'])
    X_train_lstm = pad_sequences(X_train_lstm, padding='post')

    y_train_lstm = pd.get_dummies(df_combined_nb['Sentiment']).values

    X_train_lstm, X_val_lstm, y_train_lstm, y_val_lstm = train_test_split(
        X_train_lstm, y_train_lstm, test_size=0.2, random_state=42
    )

    vocab_size = len(tokenizer.word_index) + 1
    max_length = X_train_lstm.shape[1]

    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(LSTM(128))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_lstm, y_train_lstm, validation_data=(X_val_lstm, y_val_lstm), epochs=5, batch_size=32)

    # Predict sentiment for the prediction data using the LSTM model
    X_predict_lstm = tokenizer.texts_to_sequences(df_predict_nb['Title'])
    X_predict_lstm = pad_sequences(X_predict_lstm, padding='post', maxlen=max_length)

    y_predict_lstm = model.predict(X_predict_lstm)
    y_predict_lstm = np.argmax(y_predict_lstm, axis=1)
    y_predict_lstm = ['Negative' if s == 0 else 'Neutral' if s == 1 else 'Positive' for s in y_predict_lstm]

    # Create a pandas DataFrame for prediction data using the LSTM model
    predict_data_lstm = {
        'Title': titles_predict,
        'Sentiment': y_predict_lstm,
        'Post Date': pd.to_datetime(post_dates_predict, unit='s')
    }
    df_predict_lstm = pd.DataFrame(predict_data_lstm)

    # Combine the training and prediction DataFrames using the LSTM model
    df_combined_lstm = pd.concat([df_train, df_predict_lstm], ignore_index=True)

    return df_combined_nb, df_combined_svm, df_combined_lr, df_combined_lstm, \
           accuracy_nb, accuracy_svm, accuracy_lr

def train_h2o_automl(X_train_h2o, y_train_h2o):
    # Convert the data to an H2O Frame
    train_data = pd.concat([X_train_h2o, y_train_h2o], axis=1)
    h2o_train_data = h2o.H2OFrame(train_data)

    # Identify the target and features
    target_col = "Sentiment"
    features_cols = h2o_train_data.columns.remove(target_col)

    # Prepare the AutoML run
    aml = H2OAutoML(max_models=10, seed=1)
    aml.train(x=features_cols, y=target_col, training_frame=h2o_train_data)

    # Get the best model
    best_model = aml.leader

    return best_model

def plot_sentiment_analysis_results(df_combined_nb, df_combined_svm, df_combined_lr, df_combined_lstm):
    # Plot the sentiment analysis results for Naive Bayes
    sentiment_counts_nb = df_combined_nb[df_combined_nb['Sentiment'].isin(['Positive', 'Negative', 'Neutral'])]['Sentiment'].value_counts()
    sentiment_counts_nb.plot(kind='bar', xlabel='Sentiment', ylabel='Count', title='Naive Bayes Sentiment Analysis Results')
    plt.show()

    # Plot the sentiment analysis results for SVM
    sentiment_counts_svm = df_combined_svm[df_combined_svm['Sentiment'].isin(['Positive', 'Negative', 'Neutral'])]['Sentiment'].value_counts()
    sentiment_counts_svm.plot(kind='bar', xlabel='Sentiment', ylabel='Count', title='SVM Sentiment Analysis Results')
    plt.show()

    # Plot the sentiment analysis results for Logistic Regression
    sentiment_counts_lr = df_combined_lr[df_combined_lr['Sentiment'].isin(['Positive', 'Negative', 'Neutral'])]['Sentiment'].value_counts()
    sentiment_counts_lr.plot(kind='bar', xlabel='Sentiment', ylabel='Count', title='Logistic Regression Sentiment Analysis Results')
    plt.show()

    # Plot the sentiment analysis results for LSTM
    sentiment_counts_lstm = df_combined_lstm[df_combined_lstm['Sentiment'].isin(['Positive', 'Negative', 'Neutral'])]['Sentiment'].value_counts()
    sentiment_counts_lstm.plot(kind='bar', xlabel='Sentiment', ylabel='Count', title='LSTM Sentiment Analysis Results')
    plt.show()

def main():
    try:
        subreddit_name = "france"
        num_posts_train = 1000
        num_posts_predict = 2000

        # Retrieve data and perform sentiment analysis
        df_combined_nb, df_combined_svm, df_combined_lr, df_combined_lstm, \
        accuracy_nb, accuracy_svm, accuracy_lr = get_posts_data(subreddit_name, num_posts_train, num_posts_predict)

        print("Naive Bayes Accuracy:", accuracy_nb)
        print("SVM Accuracy:", accuracy_svm)
        print("Logistic Regression Accuracy:", accuracy_lr)

        # Start an MLFlow experiment
        mlflow.set_experiment("SentimentAnalysis")

        # Retrieve data and perform sentiment analysis
        with mlflow.start_run():
            # Log the model parameters and metrics
            params = {
                "subreddit_name": subreddit_name,
                "num_posts_train": num_posts_train,
                "num_posts_predict": num_posts_predict,
                "vectorizer": "TfidfVectorizer",
                "LSTM_embedding_dim": 100,
                "LSTM_units": 128,
                "LSTM_epochs": 5,
                "LSTM_batch_size": 32,
            }
            log_model_params(params)

            log_model_metrics(accuracy_nb)
            log_model_metrics(accuracy_svm)
            log_model_metrics(accuracy_lr)

            # Train and predict using H2O's AutoML
            X_train_h2o = df_combined_nb.drop(columns=['Sentiment', 'Post Date'])
            y_train_h2o = df_combined_nb['Sentiment']

            best_model = train_h2o_automl(X_train_h2o, y_train_h2o)

            # Preprocess the prediction data for H2O AutoML
            X_predict_h2o = df_combined_svm.drop(columns=['Sentiment', 'Post Date'])

            # Retrieve post dates from the combined dataframe
            post_dates_predict = df_combined_svm['Post Date']

            # Predict sentiment using H2O's AutoML
            h2o_predict = h2o.H2OFrame(X_predict_h2o)
            h2o_predictions = best_model.predict(h2o_predict).as_data_frame()['predict'].values.tolist()

            # Log the trained models as artifacts
            # Log Naive Bayes model as pickle
            model_dir = "nb_model"
            os.makedirs(model_dir, exist_ok=True)
            model_path = os.path.join(model_dir, "model.pkl")
            joblib.dump(classifier_nb, model_path)
            mlflow.log_artifact(model_dir)

            # Create a pandas DataFrame for prediction data using H2O AutoML
            predict_data_h2o = {
                'Title': df_combined_svm['Title'],  # Use the title column from the dataframe
                'Sentiment': h2o_predictions,
                'Post Date': pd.to_datetime(post_dates_predict, unit='s')
            }
            df_predict_h2o = pd.DataFrame(predict_data_h2o)

            # Combine the training and prediction DataFrames using H2O AutoML
            df_combined_h2o = pd.concat([df_combined_svm, df_predict_h2o], ignore_index=True)

            # Log the trained models as artifacts
            # Save the trained sklearn models using joblib
            joblib.dump(classifier_nb, "classifier_nb.pkl")
            joblib.dump(classifier_svm, "classifier_svm.pkl")
            joblib.dump(classifier_lr, "classifier_lr.pkl")


            # Plot sentiment analysis results
            plot_sentiment_analysis_results(df_combined_nb, df_combined_svm, df_combined_lr, df_combined_lstm)

    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    main()







#def get_posts_data(subreddit_name, num_posts):
    # Credentials
#    reddit = praw.Reddit(client_id='UEJ25LF2qfVqzplDerE3tw',
#                         client_secret='lS3wj-eQEAdXgJVGHhI3xzQjBxVq6Q',
#                         user_agent='SentimentAnalysis271197')

    # Retrieve the top posts from the subreddit
        # Retrieve the top posts from the subreddit
#    subreddit = reddit.subreddit(subreddit_name)

    # Initialize lists to store data
#    titles = []
#    scores = []
#    urls = []
#    sentiments = []
#    sentiment_labels = []

    # Process and analyze sentiment for the posts
#    count = 0
#    batch_size = 900  # Adjust batch size as needed to avoid the limit exceeded issue
#    num_batches = num_posts // batch_size
#    remaining_posts = num_posts % batch_size

#    for batch in range(num_batches):
#        posts = subreddit.new(limit=batch_size)
#        for post in posts:
#            title = post.title
#            score = post.score
#            url = post.url

            # Perform sentiment analysis using TextBlob
#            analysis = TextBlob(title)
#            sentiment = analysis.sentiment.polarity

            # Append data to lists
#            titles.append(title)
#            scores.append(score)
#            urls.append(url)
#            sentiments.append(sentiment)

            # Print sentiment analysis result
#            if sentiment > 0:
#                sentiment_labels.append('Positive')
#            elif sentiment == 0:
#                sentiment_labels.append('Neutral')
#            else:
#                sentiment_labels.append('Negative')

#            count += 1
#            if count >= num_posts:
#                break

    # Process remaining posts if any
#    if remaining_posts > 0:
#        posts = subreddit.new(limit=remaining_posts)
#        for post in posts:
#            title = post.title
#            score = post.score
#            url = post.url

            # Perform sentiment analysis using TextBlob
#            analysis = TextBlob(title)
#            sentiment = analysis.sentiment.polarity

            # Append data to lists
#            titles.append(title)
#            scores.append(score)
#            urls.append(url)
#            sentiments.append(sentiment)

            # Print sentiment analysis result
#            if sentiment > 0:
#                sentiment_labels.append('Positive')
#            elif sentiment == 0:
#                sentiment_labels.append('Neutral')
#            else:
#                sentiment_labels.append('Negative')

#            count += 1
#            if count >= num_posts:
#                break

    # Create a pandas DataFrame
#    data = {
#        'Title': titles,
#        'Score': scores,
#        'URL': urls,
#        'Sentiment': sentiments,
#        'Sentiment Label': sentiment_labels
#    }

#    df = pd.DataFrame(data)
#    return df

#subreddit_name = "nike"
#num_posts = 100000  # Increase the number of posts to retrieve

#df = get_posts_data(subreddit_name, num_posts)
#print(df)
# Load your own labeled dataset with sentiment annotations
# X should contain the titles and y should contain the corresponding sentiment labels

# Training and testing dataset split
#X_train, X_test, y_train, y_test = train_test_split(df['Title'], df['Sentiment Label'], test_size=0.2, random_state=42)

# Creating a CountVectorizer to convert text into numerical features
#vectorizer = CountVectorizer()
#X_train_vectorized = vectorizer.fit_transform(X_train)

# Training the Naive Bayes classifier
#nb_classifier = MultinomialNB()
#nb_classifier.fit(X_train_vectorized, y_train)

# Transforming the test data using the same vectorizer
#X_test_vectorized = vectorizer.transform(X_test)

# Predicting sentiment on the test data
#y_pred = nb_classifier.predict(X_test_vectorized)

# Evaluating the performance of the classifier (e.g., accuracy, precision, recall, F1-score)
#accuracy = (y_pred == y_test).mean()
#print("Accuracy:", accuracy)

#print(df)



#consumer_key = "GpJQBBQaAkJBprgM4UKtta48k"
#consumer_secret = "2h0tUtoR9PL02f2mc6UGcfavS9GZnXbROyh8qKNKaemWogp5gl"
#access_token = "1463812152905453570-UQ0oWFqUMrmoy51WGp4YzeJbmzG61G"
#access_token_secret = "hjAwuMgKu2lhbhFGQosAfW8eaIrukFsfJnuaBih1EX8Ub"

#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)
#api = tweepy.API(auth)

#search_query = "nike"

# Function to perform sentiment analysis using TextBlob
#def analyze_sentiment(text):
 #   analysis = TextBlob(text)
 #   sentiment = analysis.sentiment.polarity
 #   if sentiment > 0:
 #       return 'Positive'
 #   elif sentiment == 0:
 #       return 'Neutral'
 #   else:
 #       return 'Negative'

# Function to process tweets
#def process_tweets(tweets):
#    for tweet in tweets:
#        print("Tweet:", tweet.text)
#        print("Sentiment:", analyze_sentiment(tweet.text))
#        print("-" * 50)

# Perform the search and process the tweets
#tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(10)
#process_tweets(tweets)

Found existing installation: pydantic 1.10.12
Uninstalling pydantic-1.10.12:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/pydantic-1.10.12.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pydantic/*
Proceed (Y/n)? 